In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import cross_val_score, train_test_split


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.preprocessing import PowerTransformer
from itertools import combinations
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
import numpy as np

import seaborn as sns



In [50]:
df = pd.read_csv("po2_data.csv")
df = df.drop(['subject#', 'test_time'], axis=1)
# column_relocation = df['total_updrs']
# del df['total_updrs']
# df.insert(loc=18, column='total_updrs', value=column_relocation)
male_x = df.query('sex == 0')

male_x = male_x.drop('sex', axis=1)
female_x = df.query('sex == 1')
female_x = female_x.drop('sex', axis=1)

# Separate explanatory variables (x) from the response variable (y)
# x = df.iloc[:, : -2]

# remove multicollinearity
# x = x.drop(['subject#','shimmer(apq3)', 'shimmer(%)', 'shimmer(apq5)', 'shimmer(apq11)', 'hnr'], axis=1)
print(female_x)
print(df.columns)


      age  jitter(%)  jitter(abs)  jitter(rap)  jitter(ppq5)  jitter(ddp)  \
12     68    0.00835     0.000043      0.00444       0.00431      0.01333   
13     68    0.01374     0.000082      0.00701       0.00654      0.02104   
14     68    0.01194     0.000068      0.00630       0.00800      0.01889   
15     68    0.00812     0.000044      0.00375       0.00410      0.01124   
16     68    0.00316     0.000013      0.00097       0.00120      0.00291   
...   ...        ...          ...          ...           ...          ...   
5768   68    0.00795     0.000043      0.00390       0.00400      0.01170   
5769   68    0.00751     0.000041      0.00380       0.00291      0.01140   
5770   68    0.01286     0.000069      0.00781       0.00668      0.02342   
5771   68    0.00442     0.000020      0.00196       0.00201      0.00589   
5772   68    0.00415     0.000018      0.00160       0.00210      0.00481   

      shimmer(%)  shimmer(abs)  shimmer(apq3)  shimmer(apq5)  shimmer(apq11

In [51]:
male_y_1 = male_x.iloc[:, -2]
male_y_2 = male_x.iloc[:, -1]
female_y_1 = female_x.iloc[:, -2]
female_y_2 = female_x.iloc[:, -1]

female_x_1 = female_x.iloc[:, :-2]
female_x_2 = female_x.iloc[:, :-2]
male_x_1 = male_x.iloc[:, :-2]
male_x_2 = male_x.iloc[:, :-2]



# Assuming you have your features (X) and target variable (y) prepared

def cross_validation(X, y):
    # Define a range of test set sizes
    test_size_range = [0.1, 0.2, 0.3, 0.4, 0.5]

    # Initialize an empty list to store mean performance scores
    mean_scores = []

    for test_size in test_size_range:
        # Initialize an empty list to store cross-validation scores for each fold
        fold_scores = []

        # Perform k-fold cross-validation for each test set size
        k = 5  # Number of folds
        for _ in range(k):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
            model = LinearRegression()
            model.fit(X_train, y_train)
            scores = model.score(X_test, y_test)  # You can use any appropriate performance metric here
            fold_scores.append(scores)

        # Calculate the mean score for this test set size
        mean_score = np.mean(fold_scores)
        mean_scores.append(mean_score)

    # Choose the best test set size based on the mean performance scores
    best_test_size = test_size_range[np.argmax(mean_scores)]
    print(best_test_size)
        
    return best_test_size


ratio_1 = cross_validation(male_x_1, male_y_1)
ratio_2 = cross_validation(male_x_2, male_y_2)


def test(x_1, x_2, y_1, y_2, ratio_1=0.4, ratio_2=0.4):
    x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_1, y_1, test_size=ratio_1, random_state=0)
    x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(x_2, y_2, test_size=ratio_2, random_state=0)


    # Build a linear regression model
    model_1 = LinearRegression()

    # Train (fit) the linear regression model using the training data
    model_1.fit(x_train_1, y_train_1)

    # print(f"Intercept: {model_1.intercept_}")
    # print(f"Coefficient: {model_1.coef_}")


    model_2 = LinearRegression()

    # Train (fit) the linear regression model using the training data
    model_2.fit(x_train_2, y_train_2)

    print(f"Intercept: {model_2.intercept_}")
    print(f"Coefficient: {model_2.coef_}")

    # Use linear regression to predict the values of (y) in the test set
    # based on the values of x in the test set
    y_pred_1 = model_1.predict(x_test_1)
    y_pred_2 = model_2.predict(x_test_2)

    # Optional: Show the predicted values of (y) next to the actual values of (y)
    df_pred_1 = pd.DataFrame({"Actual": y_test_1, "Predicted": y_pred_1})
    df_pred_2 = pd.DataFrame({"Actual": y_test_2, "Predicted": y_pred_2})
    print(x_train_2)
    print(df_pred_2)

    # Compute standard performance metrics of the linear regression:

    # Mean Absolute Error
    mae = metrics.mean_absolute_error(y_test_1, y_pred_1)
    # Mean Squared Error
    mse = metrics.mean_squared_error(y_test_1, y_pred_1)
    # Root Mean Square Error
    rmse =  math.sqrt(metrics.mean_squared_error(y_test_1, y_pred_1))
    # Normalised Root Mean Square Error
    y_max = y_test_1.max()
    y_min = y_test_1.min()
    rmse_norm = rmse / (y_max - y_min)
    # R-Squared
    r_2 = metrics.r2_score(y_test_1, y_pred_1)
    
    print("MAE: ", mae)
    print("MSE: ", mse)
    print("RMSE: ", rmse)
    print("RMSE (Normalised): ", rmse_norm)
    print("R_squared: ",r_2)

    
    mae = metrics.mean_absolute_error(y_test_2, y_pred_2)
    # Mean Squared Error
    mse = metrics.mean_squared_error(y_test_2, y_pred_2)
    # Root Mean Square Error
    rmse =  math.sqrt(metrics.mean_squared_error(y_test_2, y_pred_2))
    # Normalised Root Mean Square Error
    y_max = y_test_2.max()
    y_min = y_test_2.min()
    rmse_norm = rmse / (y_max - y_min)

    # R-Squared
    r_2 = metrics.r2_score(y_test_2, y_pred_2)

    print("MAE: ", mae)
    print("MSE: ", mse)
    print("RMSE: ", rmse)
    print("RMSE (Normalised): ", rmse_norm)
    print("R_squared: ",r_2)

    


test(male_x_1, male_x_2, male_y_1, male_y_2, ratio_1, ratio_2)


0.3
0.3
Intercept: 35.409642357610366
Coefficient: [ 3.03618212e-01  3.18424943e+02 -8.52756381e+04 -6.36606288e+04
 -1.47834390e+03  2.16990340e+04  2.21936646e+02  1.82299245e+00
 -4.96630782e+03 -3.81966902e+01 -1.91487734e+01  1.52427599e+03
 -3.46732484e+01 -4.06725455e-01  2.79769827e+00 -3.18378037e+01
  2.82075786e+01]
      age  jitter(%)  jitter(abs)  jitter(rap)  jitter(ppq5)  jitter(ddp)  \
489    60    0.00454     0.000042      0.00245       0.00266      0.00734   
1255   65    0.00229     0.000011      0.00133       0.00118      0.00398   
1533   57    0.00338     0.000018      0.00139       0.00190      0.00417   
743    49    0.00410     0.000033      0.00161       0.00176      0.00482   
477    49    0.00435     0.000048      0.00155       0.00144      0.00465   
...   ...        ...          ...          ...           ...          ...   
1238   66    0.00541     0.000042      0.00192       0.00302      0.00575   
4817   55    0.01057     0.000073      0.00521       0.

In [52]:
"""
APPLY Z-SCORE STANDARDISATION
"""
scaler = StandardScaler()

# Drop the previously added constant

# Apply z-score standardisation to all explanatory variables
std_x_1 = scaler.fit_transform(male_x_1.values)
std_x_2 = scaler.fit_transform(male_x_2.values)

# Restore the column names of each explanatory variable
std_x_1_df = pd.DataFrame(std_x_1, index=male_x_1.index, columns=male_x_1.columns)
std_x_2_df = pd.DataFrame(std_x_2, index=male_x_2.index, columns=male_x_2.columns)
ratio_1 = cross_validation(male_x_1, male_y_1)
ratio_2 = cross_validation(male_x_2, male_y_2)

test(std_x_1_df, std_x_2_df, male_y_1, male_y_2, ratio_1, ratio_2)


0.3
0.3
Intercept: 29.7234502553439
Coefficient: [   2.38376391    1.37800041   -2.80388297 -162.77572404   -3.57086984
  166.44892999    4.2189343     0.31009743  -53.94599844   -0.44110665
   -0.28975973   49.6716825    -0.88367482   -1.42066278    0.25342516
   -2.33203595    2.32061033]
           age  jitter(%)  jitter(abs)  jitter(rap)  jitter(ppq5)  \
489  -0.645835  -0.327759    -0.178581    -0.146485     -0.163052   
1255 -0.008989  -0.847684    -1.125351    -0.584511     -0.775773   
1533 -1.027942  -0.595809    -0.903029    -0.561045     -0.477693   
743  -2.046896  -0.429434    -0.438920    -0.475004     -0.535653   
477  -2.046896  -0.371664     0.013936    -0.498470     -0.668133   
...        ...        ...          ...          ...           ...   
1238  0.118381  -0.126722    -0.181623    -0.353765     -0.014011   
4817 -1.282681   1.065638     0.778833     0.932935      0.619410   
2424 -0.263727   0.289218     0.124945     0.530108      0.147449   
3851  1.137335  -0

In [53]:
df = pd.concat([std_x_1_df, female_y_1], axis=1)
# df = pd.concat([std_x_1_df, female_y_1], axis=1)

print(df.corr())
# print(std_x_df)



var = ['shimmer(dda)', 'jitter(%)']
# var = ['shimmer(dda)', 'jitter(rap)','rpde']
# var = ['jitter(%)', 'shimmer(dda)']
# for v in var:
for v in var:
    zero_or_negative_mask = std_x_1_df[v] <= 0
    std_x_1_df.loc[zero_or_negative_mask, v] = 1e-6  # Replace with a small positive value
    std_x_1_df[v + "_log"] = std_x_1_df[v].apply(np.log)
    # plt.figure(figsize=(20,10))

    # plt.subplot(1,2,1)
    # plt.scatter(df["jitter(%)"], df["motor_updrs"], color="green")
    # plt.title("Original jitter(%)")
    # plt.xlabel("jitter(%)")
    # plt.ylabel("motor updrs")
    # # plt.plot([0,40],[30,0])

    # plt.subplot(1,2,2)
    # plt.scatter(df["LOGSTAT"], df["MV"], color="red")
    # plt.title("Log Transformed LSTAT")
    # plt.xlabel("LOGSTAT")
    # plt.ylabel("MV")
    # plt.plot([0,4],[50,0])

    plt.show()

    std_x_1_df = std_x_1_df.drop([v], axis=1)

    zero_or_negative_mask = std_x_2_df[v] <= 0
    std_x_2_df.loc[zero_or_negative_mask, v] = 1e-6  # Replace with a small positive value
    std_x_2_df[v + "_log"] = std_x_2_df[v].apply(np.log)
    std_x_2_df = std_x_2_df.drop([v], axis=1)

ratio_1 = cross_validation(std_x_1_df, male_y_1)
ratio_2 = cross_validation(std_x_2_df, male_y_2)

# print(std_x_df)
test(std_x_1_df, std_x_2_df, male_y_1, male_y_2, ratio_1, ratio_2)


                     age  jitter(%)  jitter(abs)  jitter(rap)  jitter(ppq5)  \
age             1.000000  -0.050689    -0.058528    -0.079755     -0.045268   
jitter(%)      -0.050689   1.000000     0.853016     0.981051      0.961059   
jitter(abs)    -0.058528   0.853016     1.000000     0.824973      0.748880   
jitter(rap)    -0.079755   0.981051     0.824973     1.000000      0.951717   
jitter(ppq5)   -0.045268   0.961059     0.748880     0.951717      1.000000   
jitter(ddp)    -0.079768   0.981053     0.824972     0.999999      0.951723   
shimmer(%)      0.138447   0.552605     0.505480     0.550204      0.549921   
shimmer(abs)    0.156213   0.574063     0.519207     0.564814      0.567944   
shimmer(apq3)   0.111403   0.489125     0.473022     0.504036      0.475609   
shimmer(apq5)   0.142699   0.502001     0.450812     0.494631      0.519648   
shimmer(apq11)  0.198808   0.522617     0.439834     0.493086      0.554275   
shimmer(dda)    0.111400   0.489121     0.473019    

In [54]:
scaler = PowerTransformer()

# Apply the transformer to make all explanatory variables more Gaussian-looking
std_x_1 = scaler.fit_transform(std_x_1_df.values)

# Restore column names of explanatory variables
std_x_1_df = pd.DataFrame(std_x_1, index=std_x_1_df.index, columns=std_x_1_df.columns)
std_x_2 = scaler.fit_transform(std_x_2_df.values)

# Restore column names of explanatory variables
std_x_2_df = pd.DataFrame(std_x_2, index=std_x_2_df.index, columns=std_x_2_df.columns)
ratio_1 = cross_validation(std_x_1_df, male_y_1)
ratio_2 = cross_validation(std_x_2_df, male_y_2)

test(std_x_1_df, std_x_2_df, male_y_1, male_y_2, ratio_1, ratio_2)

0.1
0.3
Intercept: 29.7703607640619
Coefficient: [ 2.72433102e+00  7.11022944e-01 -6.74664650e+01 -2.35555718e+00
  6.98363916e+01  1.10361862e+00  3.46894972e+00 -5.45496146e+00
 -8.79547561e-01 -4.36458964e-01 -2.89950462e+00 -2.57569222e+00
 -1.14443258e-02 -3.09407906e+00  2.65845588e+00  1.62203224e+00
 -7.11377573e-01]
           age  jitter(abs)  jitter(rap)  jitter(ppq5)  jitter(ddp)  \
489  -0.681387     0.104379     0.161334      0.068799     0.158366   
1255 -0.070127    -2.100505    -1.122791     -1.764411    -1.127729   
1533 -1.028011    -1.456278    -1.037332     -0.741709    -1.037460   
743  -1.904110    -0.363036    -0.740818     -0.920534    -0.745230   
477  -1.904110     0.384271    -0.819090     -1.365289    -0.819207   
...        ...          ...          ...           ...          ...   
1238  0.059015     0.099512    -0.366629      0.362908    -0.370484   
4817 -1.252882     1.108747     1.444342      1.161627     1.444338   
2424 -0.320755     0.521589     1.

In [55]:
# applying the forward stepwise selection, and the r square increase is neglecting 
def forward_stepwise_selection(X, y, ratio):
    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=ratio, random_state=0)

    selected_features = []
    best_error = np.inf  # Initialize with a large value

    while len(selected_features) < len(X.columns):
        remaining_features = [feature for feature in X.columns if feature not in selected_features]
        errors = []
        
        for feature in remaining_features:
            current_features = selected_features + [feature]
            X_train_current = X_train_1[current_features]
            X_test_current = X_test_1[current_features]
            
            # Fit a model using statsmodels
            model = sm.OLS(y_train_1, sm.add_constant(X_train_current)).fit()
            
            # Make predictions and calculate error
            y_pred = model.predict(sm.add_constant(X_test_current))
            error = mean_squared_error(y_test_1, y_pred)
            errors.append((feature, error))
        
        # Find the feature that results in the lowest error
        best_feature, best_error_for_feature = min(errors, key=lambda x: x[1])
        
        # If the new feature improves the model, add it to the selected features
        if best_error_for_feature < best_error:
            selected_features.append(best_feature)
            best_error = best_error_for_feature
        else:
            break  # If no improvement, break the loop

    return selected_features

ratio_1 = cross_validation(std_x_1_df, male_y_1)
ratio_2 = cross_validation(std_x_2_df, male_y_2)

selected_features_1 = forward_stepwise_selection(std_x_1_df, male_y_1, ratio_1)
selected_features_2 = forward_stepwise_selection(std_x_2_df, male_y_2, ratio_2)


keep_1 = selected_features_1
keep_2 = selected_features_2
print(keep_1)
print(keep_2)

# keep = ['age', 'test_time', 'jitter(%)', 'jitter(abs)', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'shimmer(dda)', 'nhr', 'hnr', 'dfa']
# ['age', 'sex', 'test_time', 'jitter(%)', 'jitter(abs)', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'shimmer(dda)', 'nhr', 'hnr', 'dfa']
std_x_1_df = std_x_1_df[[x for x in keep_1]]
std_x_2_df = std_x_2_df[[x for x in keep_2]]

ratio_1 = cross_validation(std_x_1_df, male_y_1)
ratio_2 = cross_validation(std_x_2_df, male_y_2)


test(std_x_1_df, std_x_2_df, male_y_1, male_y_2, ratio_1, ratio_2)


0.1
0.3
['age', 'dfa', 'ppe', 'jitter(abs)', 'hnr', 'shimmer(apq3)', 'shimmer(%)', 'nhr', 'shimmer(abs)', 'shimmer(dda)_log', 'jitter(rap)', 'shimmer(apq5)', 'shimmer(apq11)']
['age', 'dfa', 'ppe', 'jitter(ddp)', 'hnr', 'nhr', 'shimmer(apq3)', 'shimmer(dda)_log', 'shimmer(abs)', 'jitter(ppq5)', 'jitter(%)_log', 'rpde', 'shimmer(apq5)', 'shimmer(%)', 'shimmer(apq11)']
0.1
0.3
Intercept: 29.76752253604391
Coefficient: [ 2.71751768 -2.99681167  2.66441609  2.51400991 -2.61657251 -2.7997418
 -5.68003994  1.61575412  3.28773535 -2.05049191 -0.64676841  0.10137434
 -0.93688161  1.51072366 -0.46803965]
           age       dfa       ppe  jitter(ddp)       hnr       nhr  \
489  -0.681387  1.168580  0.063974     0.158366 -1.664886  0.751927   
1255 -0.070127 -1.232675 -3.166412    -1.127729  2.244148 -1.534730   
1533 -1.028011 -1.597335 -0.472487    -1.037460  0.507170 -0.664703   
743  -1.904110  1.029956  0.029144    -0.745230  1.006877 -0.917210   
477  -1.904110  0.834612 -1.720089    -0.8

In [56]:
# keep_1 = best_features_1
# keep_2 = best_features_2
# keep_1 = ['age', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(abs)', 'shimmer(apq3)', 'shimmer(apq11)', 'nhr', 'hnr', 'dfa', 'ppe', 'jitter(%)_log', 'shimmer(dda)_log']

# keep_2 = ['age', 'jitter(abs)', 'jitter(rap)', 'jitter(ppq5)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(abs)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'nhr', 'hnr', 'rpde', 'dfa', 'jitter(%)_log', 'shimmer(dda)_log']

print(keep_1)
print(keep_2)
# keep = ['age', 'test_time', 'jitter(%)', 'jitter(abs)', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'shimmer(dda)', 'nhr', 'hnr', 'dfa']
# ['age', 'sex', 'test_time', 'jitter(%)', 'jitter(abs)', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'shimmer(dda)', 'nhr', 'hnr', 'dfa']
std_x_1_df = std_x_1_df[[x for x in keep_1]]
std_x_2_df = std_x_2_df[[x for x in keep_2]]

ratio_1 = cross_validation(std_x_1_df, male_y_1)
ratio_2 = cross_validation(std_x_2_df, male_y_2)


test(std_x_1_df, std_x_2_df, male_y_1, male_y_2, ratio_1, ratio_2)

['age', 'dfa', 'ppe', 'jitter(abs)', 'hnr', 'shimmer(apq3)', 'shimmer(%)', 'nhr', 'shimmer(abs)', 'shimmer(dda)_log', 'jitter(rap)', 'shimmer(apq5)', 'shimmer(apq11)']
['age', 'dfa', 'ppe', 'jitter(ddp)', 'hnr', 'nhr', 'shimmer(apq3)', 'shimmer(dda)_log', 'shimmer(abs)', 'jitter(ppq5)', 'jitter(%)_log', 'rpde', 'shimmer(apq5)', 'shimmer(%)', 'shimmer(apq11)']
0.1
0.3
Intercept: 29.76752253604391
Coefficient: [ 2.71751768 -2.99681167  2.66441609  2.51400991 -2.61657251 -2.7997418
 -5.68003994  1.61575412  3.28773535 -2.05049191 -0.64676841  0.10137434
 -0.93688161  1.51072366 -0.46803965]
           age       dfa       ppe  jitter(ddp)       hnr       nhr  \
489  -0.681387  1.168580  0.063974     0.158366 -1.664886  0.751927   
1255 -0.070127 -1.232675 -3.166412    -1.127729  2.244148 -1.534730   
1533 -1.028011 -1.597335 -0.472487    -1.037460  0.507170 -0.664703   
743  -1.904110  1.029956  0.029144    -0.745230  1.006877 -0.917210   
477  -1.904110  0.834612 -1.720089    -0.819207 -0

In [57]:
std_x_1_df = sm.add_constant(std_x_1_df)
model_1 = sm.OLS(male_y_1,std_x_1_df).fit()
pred = model_1.predict(std_x_1_df)
model_details_1 = model_1.summary()
print(model_details_1)

std_x_2_df = sm.add_constant(std_x_2_df)
model_2 = sm.OLS(male_y_2,std_x_2_df).fit()
pred = model_2.predict(std_x_2_df)
model_details_2 = model_2.summary()
print(model_details_2)





                            OLS Regression Results                            
Dep. Variable:            motor_updrs   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     51.15
Date:                Thu, 05 Oct 2023   Prob (F-statistic):          3.42e-123
Time:                        10:36:37   Log-Likelihood:                -13619.
No. Observations:                3996   AIC:                         2.727e+04
Df Residuals:                    3982   BIC:                         2.735e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               21.4602      0.116  

In [58]:
# removing high p value var
# r squared increased
# std_x_1_df = std_x_1_df.drop(['jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq11)'],axis=1)

# r squared decreased for the below var -> no removal
# std_x_2_df = std_x_2_df.drop(['jitter(rap)', 'jitter(ddp)', 'shimmer(abs)', 'hnr'],axis=1)  

ratio_1 = cross_validation(std_x_1_df, male_y_1)
ratio_2 = cross_validation(std_x_2_df, male_y_2)

test(std_x_1_df, std_x_2_df, male_y_1, male_y_2, ratio_1, ratio_2)

0.1
0.3
Intercept: 29.76752253604391
Coefficient: [ 0.          2.71751768 -2.99681167  2.66441609  2.51400991 -2.61657251
 -2.7997418  -5.68003994  1.61575412  3.28773535 -2.05049191 -0.64676841
  0.10137434 -0.93688161  1.51072366 -0.46803965]
      const       age       dfa       ppe  jitter(ddp)       hnr       nhr  \
489     1.0 -0.681387  1.168580  0.063974     0.158366 -1.664886  0.751927   
1255    1.0 -0.070127 -1.232675 -3.166412    -1.127729  2.244148 -1.534730   
1533    1.0 -1.028011 -1.597335 -0.472487    -1.037460  0.507170 -0.664703   
743     1.0 -1.904110  1.029956  0.029144    -0.745230  1.006877 -0.917210   
477     1.0 -1.904110  0.834612 -1.720089    -0.819207 -0.518134 -0.294392   
...     ...       ...       ...       ...          ...       ...       ...   
1238    1.0  0.059015 -0.730221  0.809455    -0.370484  0.193981  1.294173   
4817    1.0 -1.252882  0.591689  1.275525     1.444338 -1.046442  1.404094   
2424    1.0 -0.320755  0.219205 -0.679038     1.1346

In [59]:
print(std_x_1_df.columns)
print(std_x_2_df.columns)

Index(['const', 'age', 'dfa', 'ppe', 'jitter(abs)', 'hnr', 'shimmer(apq3)',
       'shimmer(%)', 'nhr', 'shimmer(abs)', 'shimmer(dda)_log', 'jitter(rap)',
       'shimmer(apq5)', 'shimmer(apq11)'],
      dtype='object')
Index(['const', 'age', 'dfa', 'ppe', 'jitter(ddp)', 'hnr', 'nhr',
       'shimmer(apq3)', 'shimmer(dda)_log', 'shimmer(abs)', 'jitter(ppq5)',
       'jitter(%)_log', 'rpde', 'shimmer(apq5)', 'shimmer(%)',
       'shimmer(apq11)'],
      dtype='object')


In [60]:
# feature engineering
std_x_1_df['doshn'] = std_x_1_df['hnr']**2 - std_x_1_df['nhr']
std_x_2_df['doshn'] = std_x_2_df['hnr']**2 - std_x_2_df['nhr']
std_x_1_df['dshn'] = std_x_1_df['hnr']**2 / std_x_1_df['nhr']
std_x_2_df['dshn'] = std_x_2_df['hnr']**2 / std_x_2_df['nhr']



# std_x_1_df['rsj'] = std_x_1_df['shimmer(%)'] / std_x_1_df['jitter(%)']
# std_x_2_df['doshn'] = std_x_2_df['shimmer(%)']**2 - std_x_2_df['jitter(%)']
# print(std_x_1_df.head())
ratio_1 = cross_validation(std_x_1_df, male_y_1)
ratio_2 = cross_validation(std_x_2_df, male_y_2)
test(std_x_1_df, std_x_2_df, male_y_1, male_y_2, ratio_1, ratio_2)


0.1
0.3
Intercept: 30.44745278011817
Coefficient: [ 0.00000000e+00  2.62328290e+00 -3.00432086e+00  2.54588121e+00
  2.50218586e+00 -2.84112547e+00 -3.26791620e+00 -6.74378462e+00
  1.93125977e+00  3.03819307e+00 -2.58916345e+00 -3.00934586e-01
 -2.33405846e-02 -6.12800732e-01  2.65551309e+00 -8.65270707e-01
 -6.96476489e-01 -3.74281453e-03]
      const       age       dfa       ppe  jitter(ddp)       hnr       nhr  \
489     1.0 -0.681387  1.168580  0.063974     0.158366 -1.664886  0.751927   
1255    1.0 -0.070127 -1.232675 -3.166412    -1.127729  2.244148 -1.534730   
1533    1.0 -1.028011 -1.597335 -0.472487    -1.037460  0.507170 -0.664703   
743     1.0 -1.904110  1.029956  0.029144    -0.745230  1.006877 -0.917210   
477     1.0 -1.904110  0.834612 -1.720089    -0.819207 -0.518134 -0.294392   
...     ...       ...       ...       ...          ...       ...       ...   
1238    1.0  0.059015 -0.730221  0.809455    -0.370484  0.193981  1.294173   
4817    1.0 -1.252882  0.591689 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3996 entries, 0 to 5862
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   const             3996 non-null   float64
 1   age               3996 non-null   float64
 2   dfa               3996 non-null   float64
 3   ppe               3996 non-null   float64
 4   jitter(abs)       3996 non-null   float64
 5   hnr               3996 non-null   float64
 6   shimmer(apq3)     3996 non-null   float64
 7   shimmer(%)        3996 non-null   float64
 8   nhr               3996 non-null   float64
 9   shimmer(abs)      3996 non-null   float64
 10  shimmer(dda)_log  3996 non-null   float64
 11  jitter(rap)       3996 non-null   float64
 12  shimmer(apq5)     3996 non-null   float64
 13  shimmer(apq11)    3996 non-null   float64
 14  doshn             3996 non-null   float64
 15  dshn              3996 non-null   float64
dtypes: float64(16)
memory usage: 659.8 KB
None